> 整理自己對於Keras的用法以及心得，盡量詳細說明每個步驟  
> 因為中文版的Keras教學太久沒更新有一些新的方法也會整理上來，主要內容來自於[Keras官方文檔](https://keras.io/)  
> 若有任何錯誤請不令指正

# **Keras建立Model很簡單**  
使用Keras搭建model有兩種方法：  
* Sequential Model：限制較多，如：多輸入多輸出、共享layer、非線性連接layer、單layer多輸入輸出等皆不適用。  
* Functional API：靈活度極高的搭建Model方法。  

**Model通常是有向無環圖(directed acyclic graph (DAG))**  

本篇只有建立model，並無包含training、evaluation、inference

In [1]:
#載入所需的lib
import tensorflow as tf
#TensorFlow若使用GPU預設會直接佔去9成RAM，所以透過命令更改使用多少就用多少
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0


## **Sequential Model**  
使用Keras建立model就像搭積木一樣，一層一層依照自己預計的方式堆疊，完成之後就有許多便利的method使用。  
更多細節用法參考[The Sequential model](https://keras.io/guides/sequential_model/)、[Sequential API](https://keras.io/api/models/sequential/)。

In [2]:
#創建Sequential model與其中的layer

#需要給定intput shape，有些情況下可以用None取代
model = tf.keras.Sequential(
    [
        #輸入的dimension
        tf.keras.Input(shape=(16,)),
        #可選擇activation function，預設None，也可增加一層activation layer達到同樣效果
        tf.keras.layers.Dense(64, activation='relu'),
        # model或者layer皆可以給名稱
        tf.keras.layers.Dense(10, name='last_layer')
    ]
)

以上就簡單地完成了一個輸入為16-d輸出為10-d的model，可以透過`model.summary()`查看模型。

In [3]:
model.summary()
#可以看到最後一層的名稱變成"last_layer"

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1088      
_________________________________________________________________
last_layer (Dense)           (None, 10)                650       
Total params: 1,738
Trainable params: 1,738
Non-trainable params: 0
_________________________________________________________________


Sequential Model的另一種寫法

In [4]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.Input(shape=(16,)))
model2.add(tf.keras.layers.Dense(64))
model2.add(tf.keras.layers.ReLU())
model2.add(tf.keras.layers.Dense(10, name='last_layer'))

這種方法等價於上面的方式，透過`.add()`一層一層疊加上去

In [5]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1088      
_________________________________________________________________
re_lu (ReLU)                 (None, 64)                0         
_________________________________________________________________
last_layer (Dense)           (None, 10)                650       
Total params: 1,738
Trainable params: 1,738
Non-trainable params: 0
_________________________________________________________________


## **Functional API**  
使用**Functional API**也能做到與**Sequential model**相同的事情，甚至可以更靈活。  
以下使用**Functional API**方式建立與上述相同的model。  
更多詳細請查看[The Functional API](https://keras.io/guides/functional_api/)。

In [6]:
tf.keras.backend.clear_session()
#使用Functional API方式與Sequential model幾乎相同

#input layer
inputs = tf.keras.Input(shape=(16,))
#定義每一個layer
dense_1 = tf.keras.layers.Dense(64, activation='relu')
dense_2 = tf.keras.layers.Dense(10, name='last_layer')

#設定tensor的穿越路徑
x = dense_1(inputs)
x = dense_2(x)

#給定tensor的穿越路徑建立model
model = tf.keras.Model(inputs=inputs, outputs=x, name='Functional API')
model.summary()

Model: "Functional API"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                1088      
_________________________________________________________________
last_layer (Dense)           (None, 10)                650       
Total params: 1,738
Trainable params: 1,738
Non-trainable params: 0
_________________________________________________________________


透過`tf.keras.Model()`定義了model的`inputs`與`outputs`，就可以自動形成model。  
以下，另一種些法也可以搭建相同模型(一樣將activation function拉出來獨立一層)：

In [7]:
#input layer
inputs_2 = tf.keras.Input(shape=(16,))
x = tf.keras.layers.Dense(64)(inputs_2)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Dense(10)(x)

model_2 = tf.keras.Model(inputs=inputs_2, outputs=x, name='Functional API 2')
model_2.summary()

Model: "Functional API 2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1088      
_________________________________________________________________
re_lu (ReLU)                 (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 1,738
Trainable params: 1,738
Non-trainable params: 0
_________________________________________________________________


以上這四個model皆等價，個人現在習慣使用**Functional API**，因為現在的模型越來越複雜，使用**Functional API**才有辦法實現。